In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_miocardial()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.05)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (1700, 110)
5.62% missing data
Class distribution: (array([0, 1]), array([1429,  271]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': {
        'input_channels':1,
        'output_channels':1,
        'channels_skip':4,
        'down_layers':[(24, 7, 2),
                       (46, 9, 1),
                       (96, 9, 5),
                       (96, 11, 1)],
        'need_sigmoid':True,
        'need_bias':True,
        'pad':'zero',
        'downsample_mode':'stride',
        'upsample_mode':'nearest',
        'act_fun':'LeakyReLU',
        'need1x1_up':True
    },# or a list containing layers sizes
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 501, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 3.37 sec - Loss 0.233687 - ACC 75.02% - ACC Mean 75.02% - AUC 83.42% - AUC Mean 83.42% - Deter 000
Ite 00050 - 0.82 sec - Loss 0.116730 - ACC 74.43% - ACC Mean 75.17% - AUC 82.49% - AUC Mean 82.27% - Deter 037
Ite 00063 - 0.91 sec - Loss 0.107708 - ACC 74.15% - ACC Mean 74.39% - AUC 81.77% - AUC Mean 82.14% - Deter 050
Early stop ite 63, rollback to correction of ite 13, whith acc of 77.63% and auc of 84.89%
OUR METHOD RUN 1/10 - acc: 77.63% +- 0.0% - auc: 84.89% +- 0.0%
Ite 00000 - 0.87 sec - Loss 0.239321 - ACC 77.33% - ACC Mean 77.33% - AUC 81.58% - AUC Mean 81.58% - Deter 000
Ite 00050 - 0.86 sec - Loss 0.126545 - ACC 76.20% - ACC Mean 76.82% - AUC 82.98% - AUC Mean 83.13% - Deter 047
Ite 00053 - 0.89 sec - Loss 0.124028 - ACC 78.12% - ACC Mean 76.80% - AUC 83.41% - AUC Mean 83.05% - Deter 050
Early stop ite 53, rollback to correction of ite 3, whith acc of 78.13% and auc of 85.47%
OUR METHOD RUN 2/10 - acc: 78.13% +- 0.0% - auc: 85.47% +- 0.0%
Ite 00000 - 0.85 sec - Lo

## Run Median, Mean and KNN

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

median_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
knn_imputer = KNNImputer(n_neighbors=10, weights='uniform')

median_completed = median_imputer.fit_transform(data_missing_nans)
mean_completed = mean_imputer.fit_transform(data_missing_nans)
knn_completed = knn_imputer.fit_transform(data_missing_nans)

## All runs would be the same since deterministic methods
scores = utils.get_scores(median_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEDIAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
median_accs = acc
median_aucs = auc

scores = utils.get_scores(mean_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mean_accs = acc
mean_aucs = auc

scores = utils.get_scores(knn_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'KNN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
knn_accs = acc
knn_aucs = auc

MEDIAN - acc: 63.3% +- 0.0% - auc: 73.71% +- 0.0%
MEAN - acc: 76.22% +- 0.0% - auc: 80.28% +- 0.0%
KNN - acc: 67.93% +- 0.0% - auc: 77.86% +- 0.0%


## Run GAIN

In [6]:
gain_parameters = {'batch_size': 128,
                   'hint_rate': .9,
                   'alpha': 100,
                   'iterations': 5000}

gain_accs, gain_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = gain(data_missing_nans, gain_parameters)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nGAIN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    gain_accs.append(acc.mean())
    gain_aucs.append(auc.mean())
print(f'GAIN GLOBAL - acc: {round(np.array(gain_accs).mean() * 100, 2)}% +- {round(np.array(gain_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(gain_aucs).mean() * 100, 2)}% +- {round(np.array(gain_aucs).std() * 100, 2)}%')

100%|██████████| 5000/5000 [00:48<00:00, 103.80it/s]



GAIN RUN 1/10 - acc: 75.02% +- 0.98% - auc: 81.37% +- 1.9%



100%|██████████| 5000/5000 [00:47<00:00, 104.48it/s]



GAIN RUN 2/10 - acc: 74.52% +- 1.52% - auc: 81.59% +- 2.66%



100%|██████████| 5000/5000 [00:48<00:00, 102.67it/s]


...


GAIN RUN 8/10 - acc: 75.33% +- 2.35% - auc: 81.36% +- 1.46%



100%|██████████| 5000/5000 [01:02<00:00, 79.44it/s]



GAIN RUN 9/10 - acc: 74.25% +- 1.53% - auc: 81.47% +- 2.52%



100%|██████████| 5000/5000 [01:01<00:00, 81.08it/s]



GAIN RUN 10/10 - acc: 72.51% +- 2.33% - auc: 81.1% +- 1.96%

GAIN GLOBAL - acc: 74.63% +- 0.91% - auc: 81.27% +- 0.77%


## Run MICE

In [7]:
imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 73.51% +- 0.0% - auc: 80.38% +- 0.0%


## Run MISSFOREST

In [8]:
miss_accs, miss_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = MissForest(n_jobs=-1, random_state=i)
    imputed = imputer.fit_transform(data_missing_nans)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMISSFOREST RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    miss_accs.append(acc.mean())
    miss_aucs.append(auc.mean())
print(f'\nMISSFOREST GLOBAL - acc: {round(np.array(miss_accs).mean() * 100, 2)}% +- {round(np.array(miss_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(miss_aucs).mean() * 100, 2)}% +- {round(np.array(miss_aucs).std() * 100, 2)}%')

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4

MISSFOREST RUN 1/10 - acc: 73.3% +- 2.02% - auc: 79.18% +- 1.18%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5

MISSFOREST RUN 2/10 - acc: 72.53% +- 1.06% - auc: 80.31% +- 1.14%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5

MISSFOREST RUN 3/10 - acc: 73.84% +- 2.16% - auc: 80.86% +- 1.23%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4

MISSFOREST RUN 4/10 - acc: 72.69% +- 2.76% - auc: 80.62% +- 1.76%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5

MISSFOREST RUN 5/10 - acc: 72.04% +- 3.27% - auc: 80.87% +- 3.28%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5

MISSFOREST RUN 6/10 - acc: 70.98% +- 3.22% - auc: 78.63% +- 3.28%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6

MISSFOREST RUN 7/10 - acc: 72.68% +- 2.69% - auc: 80.67% +-

## Run SOFTIMPUTE

In [9]:
soft_accs, soft_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = SoftImpute(random_state=i)
    imputer.fit(data_missing_nans)
    imputed = imputer.predict(data_missing_nans)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'SOFTIMPUTE RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    soft_accs.append(acc.mean())
    soft_aucs.append(auc.mean())
print(f'\nSOFTIMPUTE GLOBAL - acc: {round(np.array(soft_accs).mean() * 100, 2)}% +- {round(np.array(soft_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(soft_aucs).mean() * 100, 2)}% +- {round(np.array(soft_aucs).std() * 100, 2)}%')

SOFTIMPUTE RUN 1/10 - acc: 74.87% +- 1.07% - auc: 81.12% +- 2.06%
SOFTIMPUTE RUN 2/10 - acc: 75.04% +- 2.34% - auc: 81.59% +- 2.82%
SOFTIMPUTE RUN 3/10 - acc: 75.47% +- 1.51% - auc: 82.62% +- 1.66%
SOFTIMPUTE RUN 4/10 - acc: 73.43% +- 1.75% - auc: 80.92% +- 1.47%
SOFTIMPUTE RUN 5/10 - acc: 76.19% +- 1.04% - auc: 79.94% +- 1.23%
SOFTIMPUTE RUN 6/10 - acc: 73.81% +- 2.93% - auc: 80.8% +- 2.35%
SOFTIMPUTE RUN 7/10 - acc: 74.56% +- 2.35% - auc: 80.92% +- 1.57%
SOFTIMPUTE RUN 8/10 - acc: 75.24% +- 0.64% - auc: 81.57% +- 2.61%
SOFTIMPUTE RUN 9/10 - acc: 74.35% +- 1.65% - auc: 81.13% +- 1.14%
SOFTIMPUTE RUN 10/10 - acc: 73.98% +- 1.55% - auc: 81.95% +- 1.45%

SOFTIMPUTE GLOBAL - acc: 74.69% +- 0.8% - auc: 81.26% +- 0.69%


## Run SINKHORN

In [10]:
sink_accs, sink_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = OTimputer(niter=500)
    imputed = imputer.fit_transform(data_missing_nans).cpu().detach().numpy()
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nSINKHORN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    sink_accs.append(acc.mean())
    sink_aucs.append(auc.mean())
print(f'\nSINKHORN GLOBAL - acc: {round(np.array(sink_accs).mean() * 100, 2)}% +- {round(np.array(sink_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(sink_aucs).mean() * 100, 2)}% +- {round(np.array(sink_aucs).std() * 100, 2)}%')

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 3.7105

SINKHORN RUN 1/10 - acc: 72.56% +- 2.3% - auc: 78.11% +- 3.64%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 3.6310

SINKHORN RUN 2/10 - acc: 74.35% +- 1.14% - auc: 80.94% +- 2.36%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 3.7249

SINKHORN RUN 3/10 - acc: 72.46% +- 2.26% - auc: 81.44% +- 1.39%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 3.8107

SINKHORN RUN 4/10 - acc: 73.24% +- 2.67% - auc: 81.07% +- 1.82%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 3.8509

SINKHORN RUN 5/10 - acc: 73.09% +- 1.85% - auc: 80.14% +- 2.83%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 3.7494

SINKHORN RUN 6/10 - acc: 72.15% +- 2.67% - auc: 78.34% +- 2.34%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 3.8780

SINKHORN RUN 7/10 - acc: 73.92% +- 2.95% - auc: 80.38% +- 1.32%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 3.8436

SINKHORN RUN 8/10 - acc: 73.15% +- 1.65% - auc: 78

## Run MIDA

In [11]:
mida_accs, mida_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = mida(data_missing_nans, num_layers=2, num_epochs=1500)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMIDA RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    mida_accs.append(acc.mean())
    mida_aucs.append(auc.mean())
print(f'\nMIDA GLOBAL - acc: {round(np.array(mida_accs).mean() * 100, 2)}% +- {round(np.array(mida_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(mida_aucs).mean() * 100, 2)}% +- {round(np.array(mida_aucs).std() * 100, 2)}%')

Stop training at epoch: 1500/1500, return best output

MIDA RUN 1/10 - acc: 72.03% +- 2.03% - auc: 79.15% +- 2.76%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 2/10 - acc: 72.87% +- 1.89% - auc: 78.55% +- 3.51%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 3/10 - acc: 72.04% +- 1.13% - auc: 79.54% +- 1.19%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 4/10 - acc: 71.41% +- 1.97% - auc: 78.55% +- 2.57%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 5/10 - acc: 72.71% +- 1.12% - auc: 79.87% +- 2.88%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 6/10 - acc: 73.4% +- 2.95% - auc: 76.55% +- 3.29%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 7/10 - acc: 73.45% +- 2.44% - auc: 80.96% +- 1.8%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 8/10 - acc: 72.1% +- 2.23% - auc: 78.94% +- 1.63%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 9/10 - acc:

## Run T-tests

In [12]:
for model, metrics in {
        'MEDIAN': {'ACC': median_accs, 'AUC': median_aucs},
        'MEAN': {'ACC': mean_accs, 'AUC': mean_aucs},
        'KNN': {'ACC': knn_accs, 'AUC': knn_aucs},
        'GAIN': {'ACC': gain_accs, 'AUC': gain_aucs},
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'MISS': {'ACC': miss_accs, 'AUC': miss_aucs},
        'SOFT': {'ACC': soft_accs, 'AUC': soft_aucs},
        'SINK': {'ACC': sink_accs, 'AUC': sink_aucs},
        'MIDA': {'ACC': mida_accs, 'AUC': mida_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MEDIAN
Metric AUC - OUR METHOD is significantly better than MEDIAN
Metric ACC - OUR METHOD is even with MEAN
Metric AUC - OUR METHOD is significantly better than MEAN
Metric ACC - OUR METHOD is significantly better than KNN
Metric AUC - OUR METHOD is significantly better than KNN
Metric ACC - OUR METHOD is significantly better than GAIN
Metric AUC - OUR METHOD is significantly better than GAIN
Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than MISS
Metric AUC - OUR METHOD is significantly better than MISS
Metric ACC - OUR METHOD is significantly better than SOFT
Metric AUC - OUR METHOD is significantly better than SOFT
Metric ACC - OUR METHOD is significantly better than SINK
Metric AUC - OUR METHOD is significantly better than SINK
Metric ACC - OUR METHOD is significantly better than MIDA
Metric AUC - OUR METHOD is s